## Connection to Thymio

In [1]:
# Connect to Thymio with ClientAsync
from tdmclient import ClientAsync, aw
client = ClientAsync()
node = await client.wait_for_node()

# Lock Thymio
aw(node.lock())

Node fe36b4a4-2452-4ab5-8d68-b28bb42f53d5

## Importation of libraries

In [2]:
from kalman_filtering.kalman_filter import *
from thymio.control import *
from computer_vision.vision import *
from path_planning.path_plan import *
from Navigation.collision_avoidance import *

await node.set_variables(motors(0, 0))

## Main script

In [3]:

u_old = 0
mu_predict_old = 0
cov_predict_old = 0
view_blocked = False
active = True
end = False
start = True
path = None
segment_index = 0
avoidance_mode = 0
obj_right = False
obstacle_pos = np.array([0, 0])

map = Map()
if P_VISION:
    print(P_SETUP)
    print("Place the Thymio and the destination, check the avoidance_modes and the corners. Then, press S.")

while True:
    map.pre_update()
    cv.waitKey(0)
    if cv.waitKey(1) & 0xFF==ord('s'):
        if map.get_found_corners():
            map.update()
            break
        elif P_VISION:
            print("USER: Must find corners before start")

if P_VISION: print(P_START)
if P_VISION: map.info()
await node.wait_for_variables()

while active == True:
    active = await stop_thymio(node)

    map.update()
    
    # State variables measurement
    # await node.wait_for_variables()
    # vel_meas = velocity_with_odometry(node, map.robot[2])

    # Filtering
    # y = np.array([map.robot[0]], [map.robot[1]], [vel_meas[0]], [vel_meas[1]])
    # x_est, cov_est = kalman_filter(y, u_old, mu_predict_old, cov_predict_old, view_blocked)
    # pose_est = x_est[0:2]
    # pose_est.append(map.robot[2])
    # error_est = cov_est[0:2]
    # error_est.append(0)

    # Global Navigation
    #create path
   
    # Global Navigation
    if start == True:
        ex_path, corners = compute_visibility_matrix(map.robot[0:2], map.destination[0:2], map.obstacles)
        map.avoidance_modes_lines = possible_lignes(ex_path, corners)
        map.target_lines = a_star_search(corners, ex_path)
        path = path_functions(map.target_lines)
        start = False

    #local nav
    u, avoidance_mode, segment_index, obj_right, obstacle_pos = await collision_avoidance(path, node, map.robot, avoidance_mode, segment_index, obj_right, obstacle_pos) # Zacharie
    print("avoidance_mode", avoidance_mode)

    if not avoidance_mode:
        u, segment_index, end = path_direction(map.robot, path, segment_index)

    await node.set_variables(motors(int(u[0]), int(u[1])))

    # u_old = u
    # cov_predict_old = cov_est
    # mu_predict_old = x_est
    #avoidance_mode_old = avoidance_mode

    if cv.waitKey(20) & 0xFF==ord('p'):
        map.vision_stop()
        break

=====[   SETUP      ]===============
Place the Thymio and the destination, check the avoidance_modes and the corners. Then, press S.
ERROR: Could not detect the four aruco corners: only detected [[2]
 [1]
 [0]
 [5]
 [4]]


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\objdetect\src\aruco\aruco_detector.cpp:872: error: (-215:Assertion failed) !_image.empty() in function 'cv::aruco::ArucoDetector::detectMarkers'


## Motors control test

In [ ]:
await node.set_variables(motors(0, 0))


In [ ]:
active = True
vel_meas = []

while active:
    await node.wait_for_variables()
    #vel_meas.append(velocity_with_odometry(node, 0)[0])
    await node.set_variables(motors(-100, 100))
    active = await stop_thymio(node)

await node.set_variables(motors(0, 0)) 

vel_meas_mean = np.mean(vel_meas[-100:])
vel_meas_var = np.var(vel_meas[-100:])
print("Velocity measured:", vel_meas)
print("Velocity mean:", vel_meas_mean)
print("Velocity variance:", vel_meas_var)

Center button pressed. Stopping the algorithm.
Velocity measured: []
Velocity mean: nan
Velocity variance: nan


c:\Users\zacbo\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\zacbo\anaconda3\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\zacbo\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3787: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\zacbo\anaconda3\Lib\site-packages\numpy\core\_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
c:\Users\zacbo\anaconda3\Lib\site-packages\numpy\core\_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [ ]:
node.var

{'_fwversion': [14, 0],
 '_id': [-27410],
 '_imot': [-109, -246],
 '_integrator': [-991, -802],
 '_productId': [8],
 '_vbat': [706, 706],
 'acc': [1, 0, 22],
 'acc._tap': [32],
 'button.backward': [0],
 'button.center': [1],
 'button.forward': [0],
 'button.left': [0],
 'button.right': [0],
 'buttons._mean': [10668, 9145, 13728, 13661, 15080],
 'buttons._noise': [69, 56, 78, 105, 84],
 'buttons._raw': [10659, 9146, 12871, 13653, 15079],
 'event.args': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'event.source': [-27410],
 'leds.bottom.left': [0, 0, 0],
 'leds.bottom.right': [0, 0, 0],
 'leds.circle': [0, 0, 0, 0, 0, 0, 0, 0],
 'leds.top': [0, 0, 0],
 'mic._mean': [261],
 'mic.intensity': [65],
 'mic.threshold': [0],
 'motor.left.pwm': [31],
 'motor.left.speed': [-68],
 'motor.left.target': [0],
 'motor.right.pwm': [272],
 'motor.right.speed': [53],
 'motor.right.target': 